In [ ]:
import numpy as np
from src.jss_lite.jss_lite import jss_lite
t=np.empty((4,1),dtype=object)
import time
import os

In [ ]:
# random choice on environments:
from multiprocessing import dummy
import numpy as np
from src.jss_lite.jss_lite import jss_lite
t=np.empty((4,1),dtype=object)
import time
import os
from random import randrange
env=jss_lite(instance_path='resources/jsp_instances/standard/ft06.txt')
state=env.get_state()

env.set_state(state)
token=False
for _ in range(1):
    state=env.reset()
    #print(state)
    done=False
    i=0
    #print(state['obs'].shape)
    #for i in range(150):
    dummy_counter=0
    #print(env.get_legal_actions(state))
    while not done:
        i+=1
        legal_action=env.get_legal_actions(state)
        action=np.random.choice(len(legal_action), 1, p=(legal_action / legal_action.sum()))[0]
        #action=randrange(env.action_space.n)
        #print(action)
        if action < env.action_space.n/2:
    

            #print("before:")
            #env.render(start_count=1,x_bar="Job",y_bar="Machine")
            state, reward, done, info=env.step(action)
            if token==True:
                token=False
               # env.render()
            #print(state['obs'].shape)

        else:
            dummy_counter+=1
            #print("dummy action")
            #env.render()
            #print(action)
            #print(i)
            print(env.get_legal_actions("obs"))
            #env.render()
            state, reward, done, info=env.step(action)  

            #print(state['obs'].shape)
            token=True
                #print("after:")
                #env.render(start_count=1,x_bar="Job",y_bar="Machine")
        #print(action)
        #tmp=(env.processed_and_max_time_job_matrix)
        #print(env.count_finished_tasks_machine_matrix)
        #print(env.production_list)
        #print(state)
        #env.render()
        if done==True:
            print(f"finished! no legal actions possible after {i+1} steps")
            #print(env.timestemp_list)
            break
    print(dummy_counter)
    #env.render(start_count=1,x_bar="Job",y_bar="Machine")



In [1]:
# load agent on environment:
from copy import deepcopy
from ray import rllib, tune
import ray
from ray.rllib.contrib.alpha_zero.core.alpha_zero_trainer import AlphaZeroTrainer
from ray.rllib.policy.sample_batch import DEFAULT_POLICY_ID
from ray.rllib.policy.policy_map import PolicyMap
from ray.rllib.evaluation.episode import MultiAgentEpisode
from ray.tune.registry import register_env
from ray.rllib.contrib.alpha_zero.models.custom_torch_models import DenseModel
#from src.jss_lite.custom_torch_models import DenseModel_activation_relu as DenseModel

#from src.jss_lite.custom_torch_models import DenseModel
from ray.rllib.models.catalog import ModelCatalog
import gym
from src.jss_lite.jss_lite import jss_lite
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_multi_instances

checkpoint_path="/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/ima_3_3_no_act/3x3_0_inst/0/checkpoint-1"
loading_instance="/Users/felix/sciebo/masterarbeit/progra/model-based_rl/resources/jsp_instances/ima/3x3x3/3x3_0_inst.json"
ray.shutdown()
ray.init()
ModelCatalog.register_custom_model("dense_model", DenseModel)    
def env_creator_variable_instance(config,env_name):
    return jssp_light_obs_wrapper_multi_instances([env_name])
def env_creator(config):
    env= jssp_light_obs_wrapper_multi_instances([loading_instance])
    return env
#tune.register_env('custom_jssp', lambda config:env_creator_variable_instance(config=config,env_name=loading_instance))
tune.register_env('custom_jssp',env_creator)

config_eval = {
    "framework": "torch",
    "disable_env_checking":True,
    "num_workers"       : 6,
    "rollout_fragment_length": 50,
    "train_batch_size"  : 500,
    "sgd_minibatch_size": 64,
    "lr"                : 0.0001,
    "explore"           :False,
    #"horizon"           : 600,
    #"soft_horizon"      : True,
    "num_sgd_iter"      : 1,
    #"horizon"           : 100,
    "mcts_config"       : {
        "puct_coefficient"   : 1.5,
        "num_simulations"    : 100,
        "temperature"        : 1,
        "dirichlet_epsilon"  : 0.20,
        "dirichlet_noise"    : 0.03,
        "argmax_tree_policy" : True,
        "add_dirichlet_noise": False,
    },
    "ranked_rewards"    : {
        "enable": True,
    },
    "model"             : {
        "custom_model": "dense_model",

    },
}


agent = AlphaZeroTrainer( config=config_eval, env='custom_jssp')
agent.load_checkpoint(checkpoint_path)
import time
length_list=[]
reward_list=[]
for _ in range(1):
    policy = agent.get_policy(DEFAULT_POLICY_ID)
    action_list=[]
    env = env_creator_variable_instance(config="config",env_name=loading_instance)

    obs = env.reset()


    episode = MultiAgentEpisode(
        PolicyMap(0,0),
        lambda _, __: DEFAULT_POLICY_ID,
        lambda: None,
        lambda _: None,
        0,
    )

    episode.user_data['initial_state'] = env.get_state()

    done = False

    while not done:
        action, _, _ = policy.compute_single_action(obs, episode=episode)
        action_list.append(action)
        #print(action_dic[action])
        obs, reward, done, _ = env.step(action)
        #print(obs)
        #env.render(render_mode='human')
        #time.sleep(0.1)
        episode.length += 1

    length_list.append(episode.length)
    reward_list.append(reward)
    print(reward)
    env.render()
    #env.close()

2022-11-11 18:20:11,655	WARNING deprecation.py:46 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!
2022-11-11 18:20:11,660	INFO trainer.py:903 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2022-11-11 18:20:22,546	INFO trainable.py:159 -- Trainable.setup took 10.896 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


ValueError: loaded state dict contains a parameter group that doesn't match the size of optimizer's group

In [2]:
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_multi_instances
from src.jss_lite.jss_lite import jss_lite
import numpy as np
#env=jss_lite(instance_path='/resources/jsp_instances/standard/la01.txt')
#print(env.observation_space)
#state=env.reset()

#legal_action_old=state['action_mask']
#instance_list=['resources/jsp_instances/standard/ft06.txt','resources/jsp_instances/standard/ft10.txt','resources/jsp_instances/standard/ft20.txt','resources/jsp_instances/standard/abz5.txt','resources/jsp_instances/standard/abz6.txt','resources/jsp_instances/standard/abz7.txt']
p1='/Users/felix/sciebo/masterarbeit/progra/model-based_rl'
p2='/Users/felix/sciebo/masterarbeit/progra/model-based_rl/resources/jsp_instances/ima/3x3x3/3x3_22_inst.json'
#env=jss_lite(instance_path=p1+'/resources/jsp_instances/standard/la01.txt')
env=jss_lite(instance_path=p2)


instance_list=['/resources/jsp_instances/standard/la01.txt','/resources/jsp_instances/standard/la02.txt','/resources/jsp_instances/standard/la03.txt','/resources/jsp_instances/standard/la04.txt']
instance_list=[p1 + s for s in instance_list]
#instance_list=['resources/jsp_instances/standard/ft06.txt']
#env=jssp_light_obs_wrapper_multi_instances(instances_list=instance_list)
reward=0
while reward < 0.99:
    done=False
    state=env.reset()
    #print(state['obs'])
    i=0
    while not done:
        legal_action=state['action_mask']
        action=np.random.choice(len(legal_action), 1, p=(legal_action / legal_action.sum()))[0]
        
        state, reward, done, info=env.step(action)
        i+=1
        if done==True:
            #print(f"finished! no legal actions possible after {i} steps")
            break

env.render()
#print(env.env.instance)
print(reward)
print(env.current_timestep)
print(state['action_mask'])
#print(env.dummy_jobs)


1.0
22
[0 0 0 0 0 0]


In [ ]:
from difflib import restore


training_folder='/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/checkpoints_tune/contrib/AlphaZero/contrib_AlphaZero_custom_jssp_f8da7_00000_0_2022-10-07_11-11-05'
for f in os.listdir(training_folder):
    if 'checkpoint' in f:
        restore_path=training_folder+'/'+f+'/checkpoint-'+str(int(f[-6:]))
        if os.path.exists(restore_path):
            print(restore_path)

In [ ]:
import os
#print(os.path.dirname(__file__))
p1='/Users/felix/sciebo/masterarbeit/progra/model-based_rl'
instance_list=['/resources/jsp_instances/standard/la01.txt','/resources/jsp_instances/standard/la02.txt','/resources/jsp_instances/standard/la03.txt','/resources/jsp_instances/standard/la04.txt','/resources/jsp_instances/standard/la05.txt']
instance_list=[p1 + s for s in instance_list]
print(instance_list)

In [ ]:
from tracemalloc import start
from ray import rllib, tune
from ray.rllib.contrib.alpha_zero.core.alpha_zero_trainer import AlphaZeroTrainer
from ray.rllib.policy.sample_batch import DEFAULT_POLICY_ID
from ray.rllib.policy.policy_map import PolicyMap
from ray.rllib.evaluation.episode import MultiAgentEpisode
from ray.tune.registry import register_env
#from ray.rllib.contrib.alpha_zero.models.custom_torch_models import DenseModel
from src.jss_lite.custom_torch_models import DenseModel_activation_relu as DenseModel
#from src.jss_lite.custom_torch_models import ConvNetModel
from ray.rllib.models.catalog import ModelCatalog
import gym
#from ray.rllib.contrib.alpha_zero.core.alpha_zero_trainer import AlphaZeroTrainer

from copy import deepcopy
import numpy as np
import pandas as pd
import os
import time
import ray
from src.jss_lite.jss_lite import jss_lite


ray.shutdown()
ModelCatalog.register_custom_model("dense_model", DenseModel)

#curr_dir=(os.path.dirname(__file__))
curr_dir='/Users/felix/sciebo/masterarbeit/progra/model-based_rl'
instance_list=['/resources/jsp_instances/standard/la01.txt','/resources/jsp_instances/standard/la02.txt','/resources/jsp_instances/standard/la03.txt','/resources/jsp_instances/standard/la04.txt','/resources/jsp_instances/standard/la05.txt']
instance_list=[curr_dir + s for s in instance_list]
instance_path=curr_dir+'/resources/jsp_instances/standard/ft06.txt'
checkpoint_path='/training_checkpoints/checkpoints_tune'

from wrapper.jssplight_wrapper import jssp_light_obs_wrapper
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_multi_instances

def env_creator(config):
    #env = jssp_light_obs_wrapper(jss_lite(instance_path=instance_path))
    env=jssp_light_obs_wrapper_multi_instances(instances_list=instance_list)
    return env
def env_creator_single(instance):
    env=jssp_light_obs_wrapper_multi_instances(instances_list=instance)
    return env

def eval_env(agent,env):
    state=env.reset()
    policy = agent.get_policy(DEFAULT_POLICY_ID)
    episode = MultiAgentEpisode(
        PolicyMap(0,0),
        lambda _, __: DEFAULT_POLICY_ID,
        lambda: None,
        lambda _: None,
        0,
    )
    episode.user_data['initial_state'] = env.get_state()
    done = False
    episode_length=0
    start_time=time.time()
    while not done:
        action, _, _ = policy.compute_single_action(state, episode=episode)
        #print(episode_length)
        #print(f"action:{action}")
        #print(f"action_mask {state['action_mask']}")
        #print(f"action mask of action:{state['action_mask'][action]}")
        state, reward, done, _ = env.step(action)
        episode.length += 1
        episode_length += 1
    
    return time.time()-start_time, episode_length, reward

# use tune to register the custom environment for the ppo trainer
tune.register_env('custom_jssp',env_creator)
#tune.register_env('custom_jssp_single',env_creator())
config = {
    "framework": "torch",
    "disable_env_checking":True,
    "num_workers"       : 6,
    "rollout_fragment_length": 50,
    "train_batch_size"  : 500,
    "sgd_minibatch_size": 64,
    "lr"                : 0.0001,
    #"explore"           :False,
    #"horizon"           : 600,
    #"soft_horizon"      : True,
    "num_sgd_iter"      : 1,
    #"horizon"           : 100,
    "mcts_config"       : {
        "puct_coefficient"   : 1.5,
        "num_simulations"    : 100,
        "temperature"        : 1.5,
        "dirichlet_epsilon"  : 0.20,
        "dirichlet_noise"    : 0.03,
        "argmax_tree_policy" : True,
        "add_dirichlet_noise": False,
    },
    "ranked_rewards"    : {
        "enable": True,
    },
    "model"             : {
        "custom_model": "dense_model",

    },
}



agent = AlphaZeroTrainer( config=config, env='custom_jssp')
#if restore_agent:
#restore_path='training_checkpoints/checkpoints_az_jsslite/checkpoint-5'
training_folder='/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/contrib_AlphaZero_custom_jssp_50160_00000_0_2022-10-17_18-46-57'
#training_folder='/home/fs608798/masterarbeit/model-based_rl/training_checkpoints/checkpoints_tune/contrib/AlphaZero/contrib_AlphaZero_custom_jssp_50160_00000_0_2022-10-17_18-46-57'
nr_checkpoints=0
for f in os.listdir(training_folder):
    if 'checkpoint' in f:
        nr_checkpoints+=1
eval_sheet={}
run=0
for f in os.listdir(training_folder):
    if 'checkpoint' in f:
        restore_path=training_folder+'/'+f+'/checkpoint-'+str(int(f[-6:]))
        if os.path.exists(restore_path):
            print(f)

            agent.load_checkpoint(restore_path)
            print("agent loaded")
            # here comes the evaluation:
            for instance in instance_list:
                print(instance)
                eval_sheet[str(f[-6:]),instance]=eval_env(agent,jssp_light_obs_wrapper_multi_instances(instances_list=[instance]))
            run+=1
            print(f"run: {run} of {nr_checkpoints} evaluated")
            df=pd.DataFrame.from_dict(eval_sheet)
            df.to_csv('eval.csv')

# generate csv    
df=pd.DataFrame.from_dict(eval_sheet)
df.to_csv('eval.csv')

In [ ]:
df=pd.DataFrame.from_dict(eval_sheet)
df.to_csv('eval.csv')

In [27]:
x=[1,2,3,4,5]
y=[2,3,4,5,6,7]
z=x+y
print(z)

[1, 2, 3, 4, 5, 2, 3, 4, 5, 6, 7]
